In [1]:
import chess
import chess.pgn
import tqdm
import random
import datetime
import time
import json
import Levenshtein

def similar(a, b):
    return Levenshtein.ratio(a, b)

In [2]:
with open("years.json") as f:
    res = ""
    for i in f:
        res += i
    ages = json.loads(res)   
    
with open("byears.json") as f:
    res = ""
    for i in f:
        res += i
    bdata = json.loads(res)   

In [3]:
def age(p):
    if p in ages.keys():
        return ages[p]
    ret = 0
    best = 0
    for key in bdata:
        if similar(key, p) > best:
            ret = bdata[key]
            best = similar(key, p)
    ages[p] = ret
    return ret

def elo_prob(diff):
    return 1 / (1 + 10 ** (-diff / 400))

In [4]:
db = []

def open_db():
    t = time.time()
    pgn = open("./dbs/o21c10.pgn")
    while True:
        headers = chess.pgn.read_headers(pgn)
        if headers is None:
            break
        db.append(headers)
    print(int(time.time() - t), "s")

In [5]:
games = []
pind = {}
rnd_num = {}

def get_games():
    tours = {}
    htours = set()
    tcnt = 0
    for ind in tqdm.tqdm(range(len(db))):
        headers = db[ind]
        if not "WhiteElo" in headers.keys() or not "BlackElo" in headers.keys() or not "White" in headers.keys() or not "Black" in headers.keys() or not "EventDate" in headers.keys() or not "PlyCount" in headers.keys() or not "Round" in headers.keys() or not "Event" in headers.keys():
            continue
        p1, p2 = headers["White"], headers["Black"]
        r1, r2 = int(headers["WhiteElo"]), int(headers["BlackElo"])
        res = headers["Result"]
        try:
            rnd = int(headers["Round"].split('.')[0])
        except:
            rnd = 1
        if res == '1-0':
            res = 1
        elif res == '1/2-1/2':
            res = 0.5
        elif res == '0-1':
            res = 0
        else:
            continue
        year = int(headers["EventDate"].split('.')[0])
        event = headers["Event"]
        if not event in htours:
            htours.add(event)
            tours[event] = tcnt
            tcnt += 1
        if tours[event] == tcnt - 1:
            rnd_num[tcnt - 1] = rnd
        else:
            rnd_num[tours[event]] = max(rnd, rnd_num[tours[event]])
        a1 = year - age(p1)
        a2 = year - age(p2)
        plies = int(headers["PlyCount"])
        try:
            y, m, d = list(int(i) for i in headers["EventDate"].split('.'))
        except:
            y, m, d = year, 1, 1
        date = datetime.datetime(y, m, d)
        if not p1 in pind.keys():
            pind[p1] = []
        if not p2 in pind.keys():
            pind[p2] = []
        pind[p1].append(len(games))
        pind[p2].append(len(games))
        games.append([p1, p2, r1, r2, res, a1, a2, plies, date, rnd, tours[event]])

In [19]:
X, y = [], []

def calc():
    global rnd_num
    lsz = 10
    rng = len(games)
    for i in tqdm.tqdm(range(rng)):
        if games[i][5] < 13 or games[i][5] > 100:
            continue
        if games[i][6] < 13 or games[i][6] > 100:
            continue
        if max(games[i][2], games[i][3]) < 2600 or min(games[i][2], games[i][3]) < 2200:
            continue
        in_pair = []
        p1, p2 = games[i][0], games[i][1]
        p1l, p2l = 0.5, 0.5
        p1g, p2g = [], []
        p1r, p2r = [], []
        results1, results2 = [], []
        p1s, p1c, p2s, p2c = 0, 0, 0, 0
        mrp1, mrp2 = 0, 0
        wbr1 = [0, 0, 0, 0]
        wbr2 = [0, 0, 0, 0]
        resa1 = [0, 0]
        resa2 = [0, 0]
        tour1 = []
        tour2 = []
        dsin1, dsin2 = 1, 1
        ngames = []
        inds1, inds2 = pind[p1][:pind[p1].index(i)], pind[p2][:pind[p2].index(i)]
        # inds = sorted(inds1 + inds2)
        inds = inds1 + inds2
        for j in inds:
            # if p1 in games[j][0] + games[j][1] or p2 in games[j][0] + games[j][1]:
            ngames.append(games[j])
        for j in inds1[::-1]:
            if games[j][10] != games[inds1[-1]][10]:
                break
            if p1 in games[j][0]:
                tour1.append(games[j][4])
            if p1 in games[j][1]:
                tour1.append(1 - games[j][4])
        for j in inds2[::-1]:
            if games[j][10] != games[inds2[-1]][10]:
                break
            if p1 in games[j][0]:
                tour2.append(games[j][4])
            if p1 in games[j][1]:
                tour2.append(1 - games[j][4])
        for j in range(len(ngames)):
            if ngames[j][0] == p1 and ngames[j][1] == p2:
                in_pair.append(ngames[j][4])
            elif ngames[j][0] == p2 and ngames[j][1] == p1:
                in_pair.append(1 - ngames[j][4])

            if ngames[j][0] == p1:
                if ngames[j][2] < ngames[j][3]:
                    wbr1[2] += 1
                    wbr1[3] += ngames[j][4]
                elif ngames[j][2] >= ngames[j][3]:
                    wbr1[0] += 1
                    wbr1[2] += ngames[j][4]
                mrp1 = max(mrp1, ngames[j][2])
                p1g.append(ngames[j][3] + 400 * (2 * ngames[j][4] - 1))
                p1r.append(ngames[j][4])
                results1.append(ngames[j][4] == 0.5)
                p1l = ngames[j][4]
                p1s += ngames[j][4]
                p1c += 1
                
            if ngames[j][1] == p2:
                if ngames[j][2] < ngames[j][3]:
                    wbr2[0] += 1
                    wbr2[1] += (1 - ngames[j][4])
                elif ngames[j][2] >= ngames[j][3]:
                    wbr2[2] += 1
                    wbr2[3] += (1 - ngames[j][4])
                mrp2 = max(mrp2, ngames[j][3])
                results2.append(ngames[j][4] == 0.5)
                p2g.append(ngames[j][2] - 400 * (2 * ngames[j][4] - 1))
                p2r.append(1 - ngames[j][4])
                p2l = 1 - ngames[j][4]
                p2s += 1 - ngames[j][4]
                p2c += 1
            
            if ngames[j][1] == p1:
                if ngames[j][2] < ngames[j][3]:
                    wbr1[0] += 1
                    wbr1[1] += (1 - ngames[j][4])
                elif ngames[j][2] >= ngames[j][3]:
                    wbr1[2] += 1
                    wbr1[3] += (1 - ngames[j][4])
                mrp1 = max(mrp1, ngames[j][3])
                results1.append(ngames[j][4] == 0.5)
                p1g.append(ngames[j][2] - 400 * (2 * ngames[j][4] - 1))
                p1r.append(1 - ngames[j][4])
                p1l = 1 - ngames[j][4]
            
            if ngames[j][0] == p2:
                if ngames[j][2] < ngames[j][3]:
                    wbr2[2] += 1
                    wbr2[3] += ngames[j][4]
                elif ngames[j][2] >= ngames[j][3]:
                    wbr2[0] += 1
                    wbr2[2] += ngames[j][4]
                mrp2 = max(mrp2, ngames[j][2])
                results2.append(ngames[j][4] == 0.5)
                p2g.append(ngames[j][3] + 400 * (2 * ngames[j][4] - 1))
                p2r.append(ngames[j][4])
                p2l = ngames[j][4]
        
        l1, l2 = 0.5, 0.5
        lm1, lm2 = 0, 0
        for j in range(len(ngames)):
            if ngames[j][0] == p1:
                if l1 == p1l:
                    resa1[0] += 1
                    resa1[1] += ngames[j][4]
                l1 = ngames[j][4]
                lm1 = ngames[j][7]
                dsin1 = (games[i][8] - ngames[j][8]).days
            if ngames[j][0] == p2:
                if l2 == p2l:
                    resa2[0] += 1
                    resa2[1] += ngames[j][4]
                l2 = ngames[j][4]
                lm2 = ngames[j][7]
                dsin2 = (games[i][8] - ngames[j][8]).days
            if ngames[j][1] == p1:
                if l1 == p1l:
                    resa1[0] += 1
                    resa1[1] += 1 - ngames[j][4]
                l1 = 1 - ngames[j][4]
                lm1 = ngames[j][7]
                dsin1 = (games[i][8] - ngames[j][8]).days
            if ngames[j][1] == p2:
                if l2 == p2l:
                    resa2[0] += 1
                    resa2[1] += 1 - ngames[j][4]
                l2 = 1 - ngames[j][4]
                lm2 = ngames[j][7]
                dsin2 = (games[i][8] - ngames[j][8]).days
        p1p = games[i][2]
        p1lr = 0.5
        if len(p1g) >= lsz:
            p1p = sum(p1g[-lsz:]) // len(p1g[-lsz:])
            p1lr = sum(p1r[-lsz:]) / len(p1r[-lsz:])
        else:
            continue
        p2p = games[i][3]
        p2lr = 0.5
        if len(p2g) >= lsz:
            p2p = sum(p2g[-lsz:]) // len(p2g[-lsz:])
            p2lr = sum(p2r[-lsz:]) / len(p2r[-lsz:])
        else:
            continue
        score = 0.5
        draw_in_pair = 0.4
        if (len(in_pair) >= 1):
            draw_in_pair = len([i for i in in_pair if i == 0.5]) / len(in_pair)
            win_in_pair = len([i for i in in_pair if i == 1]) / len(in_pair)
            lose_in_pair = len([i for i in in_pair if i == 0]) / len(in_pair)
            score = sum(in_pair) / len(in_pair)
        else:
            continue
        s1, s2 = 0.5, 0.5
        a1, a2 = 0.5, 0.5
        if p1c > 0:
            s1 = p1s / p1c
        else:
            continue
        if p2c > 0:
            s2 = p2s / p2c
        else:
            continue
        #elo_prob(games[i][2] - games[i][3]), 
        rscore1, rscore2 = 0.5, 0.5
        
        lrtg1, hrtg1, lrtg2, hrtg2 = 0.5, 0.5, 0.5, 0.5
        
        hrtg1 = wbr1[3] / wbr1[2]
        lrtg1 = wbr1[1] / wbr1[0]
        hrtg2 = wbr2[3] / wbr2[2]
        lrtg2 = wbr2[1] / wbr2[0]
        
        as1, as2 = 0.5, 0.5
        drawish1, drawish2 = 0.4, 0.4
        losing1, losing2 = 0.3, 0.3
        winning1, winning2 = 0.3, 0.3
        
        if len(results1) >= lsz:
            losing1 = len([i for i in results1[-lsz:] if i == 0]) / len(results1[-lsz:])
        else:
            continue
        if len(results2) >= lsz:
            losing2 = len([i for i in results2[-lsz:] if i == 0]) / len(results2[-lsz:])
        else:
            continue
        
        if len(results1) >= lsz:
            drawish1 = len([i for i in results1[-lsz:] if i == 0.5]) / len(results1[-lsz:])
        else:
            continue
        if len(results2) >= lsz:
            drawish2 = len([i for i in results2[-lsz:] if i == 0.5]) / len(results2[-lsz:])
        else:
            continue
        
        if len(results1) >= lsz:
            winning1 = len([i for i in results1[-lsz:] if i == 1]) / len(results1[-lsz:])
        else:
            continue
        if len(results2) >= lsz:
            winning2 = len([i for i in results2[-lsz:] if i == 1]) / len(results2[-lsz:])
        else:
            continue
        
        if resa1[0] > 0:
            as1 = resa1[1] / resa1[0]
        else:
            continue
        if resa2[0] > 0:
            as2 = resa2[1] / resa2[0]
        else:
            continue
            
        tour_res1 = 0.5
        tour_win1, tour_draw1, tour_lose1 = 0.33, 0.33, 0.33
        
        if len(tour1) > 0:
            tour_win1 = len([i for i in tour1 if i == 1]) / len(tour1)
            tour_draw1 = len([i for i in tour1 if i == 0.5]) / len(tour1)
            tour_lose1 = len([i for i in tour1 if i == 0]) / len(tour1)
            tour_res1 = sum(tour1) / len(tour1)   
        
        tour_res2 = 0.5
        tour_win2, tour_draw2, tour_lose2 = 0.33, 0.33, 0.33
        if len(tour2) > 0:
            tour_win2 = len([i for i in tour2 if i == 1]) / len(tour2)
            tour_draw2 = len([i for i in tour2 if i == 0.5]) / len(tour2)
            tour_lose2 = len([i for i in tour2 if i == 0]) / len(tour2)
            tour_res2 = sum(tour2) / len(tour2)
            
        X.append([score, win_in_pair, draw_in_pair, lose_in_pair, (min(games[i][2], games[i][3]) - 2200) / 800, elo_prob(games[i][2] - games[i][3]), elo_prob(p1p - p2p), elo_prob(mrp1 - mrp2), p1l, p2l, s1, s2, hrtg1, lrtg1, hrtg2, lrtg2, as1, as2, p1lr, p2lr, winning1, winning2, drawish1, drawish2, losing1, losing2, lm1 / 250, lm2 / 250, dsin1 / 200, dsin2 / 200, (games[i][5] - 15) / 66, (games[i][6] - 15) / 66, games[i][9], games[i][9] / rnd_num[games[i][10]], tour_res1, tour_res2, tour_win1, tour_draw1, tour_lose1, tour_win2, tour_draw2, tour_lose2])
        # X.append([score, draw_in_pair, games[i][2], games[i][3], p1p, p2p, mrp1, mrp2, p1l, p2l, s1, s2, rscore1, rscore2, as1, as2, p1lr, p2lr, winning1, winning2, drawish1, drawish2, losing1, losing2, lm1, lm2, dsin1, dsin2, games[i][5], games[i][6], games[i][9], rnd_num[games[i][10]], tour_res1, tour_res2, ])
        y.append(games[i][4])

In [7]:
open_db()

1279 s


In [8]:
get_games()

100%|███████████████████████████████████████████████████████████████████████| 1102476/1102476 [32:29<00:00, 565.62it/s]


In [20]:
calc()

100%|██████████████████████████████████████████████████████████████████████| 1102465/1102465 [10:40<00:00, 1721.86it/s]


In [21]:
if len(X) > 0:
    datax = open("new_data/X.json",'w')
    datay = open("new_data/Y.json",'w')
    json.dump(X, datax)
    json.dump(y, datay)
    datax.close()
    datay.close()

In [11]:
if len(ages.keys()) > 0:
    data = open("years.json",'w')
    json.dump(ages, data)
    data.close()

## parameters

head-to-head score ✔

win % in pair

draw % in pair ✔

lose_in_pair

least_rtg

score according to rating ✔

score according to perfomance rating ✔

score according to max rating ✔

white's last game result ✔

black's last game result ✔

white's score as white ✔

black's score as black ✔

white's score against higher\lower -rated players ✔

black's score against higher\lower -rated players ✔

white's average result after (last game result) ✔

black's average result after (last game result) ✔

white's average score ✔

black's average score ✔

white's wins % ✔

black's wins % ✔

white's draws % ✔

black's draws % ✔

white's loses % ✔

black's loses % ✔

white's last game length ✔

black's last game length ✔

days since white's last game ✔

days since black's last game ✔

white's age ✔

black's age ✔

round ✔

round / tour_len ✔

white tour results ✔

black tour results ✔

white tour wins ✔

white tour draws ✔

white tour loses ✔

black tour wins ✔

black tour draws ✔

black tour loses ✔

In [12]:
#for key in bdata:
#    if bdata[key] == 0:
#        print(key)